In [ ]:
from dotenv import load_dotenv

load_dotenv("../../../config/.env")

True

In [ ]:
from pydantic import BaseModel
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langchain.chat_models import init_chat_model
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

class WeatherResponse(BaseModel):
    conditions: str

checkpointer = InMemorySaver()

def get_weather(city: str) -> str:
    """根据给定的城市给出天气情况"""
    return f"{city}天气晴朗"

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config["configurable"].get("user_name")
    system_msg = f"你是个助手。称呼用户为{user_name}"
    return [{"role": "system", "content": system_msg}] + state["messages"]

model = init_chat_model(
    "deepseek-chat",
    temperature=0,
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt=prompt,
    checkpointer=checkpointer,
    response_format=WeatherResponse,
)

config={"configurable": {"user_name": "西恩", "thread_id": 1}}

response_1 = agent.invoke(
    {"messages": [{"role": "user", "content": "北京的天气怎么样？"}]},
    config,
)
response_1["structured_response"]

response_2 = agent.invoke(
    {"messages": [{"role": "user", "content": "伦敦的天气怎么样？"}]},
    config,
)
response_2["structured_response"]

WeatherResponse(conditions='晴朗')